In [ ]:
%cd /content
import IPython.display
import os
import sys

if not os.path.exists("/content/sd-inference-server"):
    !git clone https://github.com/arenasys/sd-inference-server.git

%cd /content/sd-inference-server
model_folder = "./models"
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    model_folder = "/content/drive/My Drive/qDiffusion/models"
    if not os.path.exists(model_folder):
        !mkdir '/content/drive/My Drive/qDiffusion' -p
        !cp -r 'models/' '/content/drive/My Drive/qDiffusion/'
except Exception as e:
    pass

if not os.path.exists("venv"):
    !curl -OJL https://huggingface.co/datasets/arenasys/qDiffusion/resolve/main/cached_venv5.tar
    !tar xf cached_venv5.tar; mv cached_venv5 venv; rm cached_venv5.tar
    !source /content/sd-inference-server/venv/bin/activate; pip3 install timm==0.6.13
    IPython.display.clear_output()

if not sys.path[0] == "/content/sd-inference-server/":
    sys.path.insert(0, "/content/sd-inference-server/venv/lib/python3.10/site-packages")
    sys.path.insert(0, "/content/sd-inference-server/")

import random
import torch
import attention
import storage
import wrapper
import string
import time
from server import Server

model_storage = storage.ModelStorage(model_folder, torch.float16, torch.float32)
params = wrapper.GenerationParameters(model_storage, torch.device("cuda"))

password = ''.join(random.SystemRandom().choice(string.ascii_letters + string.digits) for _ in range(8))

server = Server(params, "127.0.0.1", "28888", password)
server.start()

from pycloudflared import try_cloudflare
tunnel_url = try_cloudflare(port=28888, verbose=False)
print("ENDPOINT: ", tunnel_url.tunnel.replace("https", "wss"))
print("PASSWORD: ", password)

try:
    while True:
        time.sleep(1)
except:
    server.stop()
    try_cloudflare.terminate(28888)